In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, FloatType, IntegerType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf, col

spark = SparkSession.builder.master("local").appName("minLSH").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/16 18:51:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/16 18:51:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import random
from pyspark.sql.types import ArrayType, FloatType, IntegerType,StructType
from pyspark.ml.feature import HashingTF
import mmh3

# MinHashLSH

In [53]:
class minLSH:
    def __init__(self, numHashTables=5, shingleSize=5, inputCol="features", outputCol="hashes"):
        self.numHashTables = numHashTables
        self.shingleSize = shingleSize
        self.inputCol = inputCol
        self.outputCol = outputCol
        self.seed = None
        if not self.seed:
            self.seed = random.randint(0, 10000)
        random.seed(self.seed)
        a_vals = [random.randint(1, numHashTables * 10) for _ in range(numHashTables)]
        b_vals = [random.randint(0, numHashTables * 10) for _ in range(numHashTables)]
        m = 1 << 32 - 1

        self.hash_funcs = [lambda x, a=a, b=b: mmh3.hash(str(x) + str(a) + str(b)) % m
                                for a, b in zip(a_vals, b_vals)]


    def _hash_func(self, shingle, hash_func):
        hash_val = hash_func(shingle)
        return hash_val
    
    def _shingle(self, vec):
        shingles = set()
        if len(vec) < self.shingleSize:
            return list(vec)
        for i in range(len(vec) - self.shingleSize + 1):
            shingles.add(Vectors.dense(vec[i:i + self.shingleSize]))
        return list(shingles)

    def _min_hash(self, vec):
        shingles = self._shingle(vec)
        hash_values = []
        for hf in self.hash_funcs:
            min_hash = float('inf')
            for shingle in shingles:
                hash_val = self._hash_func(shingle, hf)
                if hash_val < min_hash:
                    min_hash = hash_val
            hash_values.append(min_hash)
        return Vectors.dense(hash_values)
    
  
    def fit(self, dataset):
        return self

    def transform(self, dataset):
        min_hash_udf = udf(self._min_hash, VectorUDT())
        dataset = dataset.withColumn(
            self.outputCol, min_hash_udf(col(self.inputCol)))
        return dataset

    def approxSimilarityJoin(self, datasetA, datasetB, threshold):
        jaccard_similarity = udf(lambda x, y: len(
            set(x).intersection(set(y))) / len(set(x).union(set(y))), FloatType())

        for col_name in datasetA.columns:
            datasetA = datasetA.withColumnRenamed(col_name, col_name + "_1")
        for col_name in datasetB.columns:
            datasetB = datasetB.withColumnRenamed(col_name, col_name + "_2")

        joined = datasetA.crossJoin(datasetB)
        joined = joined.withColumn("jaccard_similarity", jaccard_similarity(
            joined[self.outputCol + "_1"], joined[self.outputCol + "_2"]))
        result = joined.filter(joined["jaccard_similarity"] >= threshold)
        return result



In [4]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

In [54]:
data = [(0, Vectors.dense([1.0, 1.0])),
        (1, Vectors.dense([1.0, 2.0])),
        (2, Vectors.dense([3.0, 2.0])),
        (3, Vectors.dense([2.0, 2.0])),
        (4, Vectors.dense([2.0, 3.0])),
        (5, Vectors.dense([3.0, 2.0])),
        ]

df = spark.createDataFrame(data, ["id", "features"])
min_lsh = minLSH(numHashTables=5, inputCol="features", outputCol="hashes")
min_lsh_model = min_lsh.fit(df)
transformed_df = min_lsh_model.transform(df)
transformed_df.show()
datasetA = transformed_df.filter(df['id'] <= 2)
datasetB = transformed_df.filter(df['id'] >= 3)
print(transformed_df.select("hashes").take(1))
result = min_lsh_model.approxSimilarityJoin(datasetA, datasetB, threshold=0)
result.show()


+---+---------+--------------------+
| id| features|              hashes|
+---+---------+--------------------+
|  0|[1.0,1.0]|[1.666456792E9,5....|
|  1|[1.0,2.0]|[1.666456792E9,5....|
|  2|[3.0,2.0]|[3.6419452E8,8.85...|
|  3|[2.0,2.0]|[1.874009501E9,8....|
|  4|[2.0,3.0]|[3.6419452E8,8.85...|
|  5|[3.0,2.0]|[3.6419452E8,8.85...|
+---+---------+--------------------+

[Row(hashes=DenseVector([1666456792.0, 547096464.0, 492682732.0, 408193507.0, 339579175.0]))]
23/04/16 19:51:57 WARN ExtractPythonUDFFromJoinCondition: The join condition:(<lambda>(hashes_1#7981, hashes_2#7993)#8005 >= 0.0) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


+----+----------+--------------------+----+----------+--------------------+------------------+
|id_1|features_1|            hashes_1|id_2|features_2|            hashes_2|jaccard_similarity|
+----+----------+--------------------+----+----------+--------------------+------------------+
|   0| [1.0,1.0]|[1.666456792E9,5....|   3| [2.0,2.0]|[1.874009501E9,8....|               0.0|
|   0| [1.0,1.0]|[1.666456792E9,5....|   4| [2.0,3.0]|[3.6419452E8,8.85...|               0.0|
|   0| [1.0,1.0]|[1.666456792E9,5....|   5| [3.0,2.0]|[3.6419452E8,8.85...|               0.0|
|   1| [1.0,2.0]|[1.666456792E9,5....|   3| [2.0,2.0]|[1.874009501E9,8....|               0.0|
|   1| [1.0,2.0]|[1.666456792E9,5....|   4| [2.0,3.0]|[3.6419452E8,8.85...|               0.0|
|   1| [1.0,2.0]|[1.666456792E9,5....|   5| [3.0,2.0]|[3.6419452E8,8.85...|               0.0|
|   2| [3.0,2.0]|[3.6419452E8,8.85...|   3| [2.0,2.0]|[1.874009501E9,8....|        0.11111111|
|   2| [3.0,2.0]|[3.6419452E8,8.85...|   4| [2.0,3

In [36]:
train = spark.read.csv('./mnist/mnist_train.csv', header=True, inferSchema=True)
columns = train.columns[1:]
# 合并为 vector
assembler = VectorAssembler(inputCols=columns, outputCol='features')
train = assembler.transform(train)
train = train.drop(*columns)
# 改为 DenseVector
toDense = lambda v: Vectors.dense(v.toArray())
toDenseUdf = udf(toDense, VectorUDT())
train = train.withColumn('features', toDenseUdf('features'))


In [6]:
train.select('features').show(5, truncate=False)

23/04/16 17:27:12 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Train

In [43]:
image_df = spark.createDataFrame(train.take(10), ["id", "features"])
min_lsh = minLSH(numHashTables=300, shingleSize=20, inputCol="features", outputCol="hashes")
min_lsh_model = min_lsh.fit(image_df)
transformed_df = min_lsh_model.transform(image_df)
transformed_df.show()

+---+--------------------+--------------------+
| id|            features|              hashes|
+---+--------------------+--------------------+
|  5|[0.0,0.0,0.0,0.0,...|[6809216.0,703145...|
|  0|[0.0,0.0,0.0,0.0,...|[4740876.0,170893...|
|  4|[0.0,0.0,0.0,0.0,...|[1.3840089E7,1371...|
|  1|[0.0,0.0,0.0,0.0,...|[2681899.0,612072...|
|  9|[0.0,0.0,0.0,0.0,...|[7251994.0,1.6084...|
|  2|[0.0,0.0,0.0,0.0,...|[2337324.0,940756...|
|  1|[0.0,0.0,0.0,0.0,...|[4841990.0,1.2461...|
|  3|[0.0,0.0,0.0,0.0,...|[1.9987955E7,2269...|
|  1|[0.0,0.0,0.0,0.0,...|[1.3771277E7,9414...|
|  4|[0.0,0.0,0.0,0.0,...|[9852210.0,100924...|
+---+--------------------+--------------------+



In [45]:
result = min_lsh_model.approxSimilarityJoin(transformed_df, transformed_df, threshold=0)
result.where("id_1=1").show()

23/04/16 19:38:32 WARN ExtractPythonUDFFromJoinCondition: The join condition:(<lambda>(hashes_1#7661, hashes_2#7673)#7685 >= 0.0) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


+----+--------------------+--------------------+----+--------------------+--------------------+------------------+
|id_1|          features_1|            hashes_1|id_2|          features_2|            hashes_2|jaccard_similarity|
+----+--------------------+--------------------+----+--------------------+--------------------+------------------+
|   1|[0.0,0.0,0.0,0.0,...|[2681899.0,612072...|   5|[0.0,0.0,0.0,0.0,...|[6809216.0,703145...|               0.0|
|   1|[0.0,0.0,0.0,0.0,...|[2681899.0,612072...|   0|[0.0,0.0,0.0,0.0,...|[4740876.0,170893...|       0.001669449|
|   1|[0.0,0.0,0.0,0.0,...|[2681899.0,612072...|   4|[0.0,0.0,0.0,0.0,...|[1.3840089E7,1371...|       0.001669449|
|   1|[0.0,0.0,0.0,0.0,...|[2681899.0,612072...|   1|[0.0,0.0,0.0,0.0,...|[2681899.0,612072...|               1.0|
|   1|[0.0,0.0,0.0,0.0,...|[2681899.0,612072...|   9|[0.0,0.0,0.0,0.0,...|[7251994.0,1.6084...|               0.0|
|   1|[0.0,0.0,0.0,0.0,...|[2681899.0,612072...|   2|[0.0,0.0,0.0,0.0,...|[23373

In [44]:
hashes2 = result.select("hashes_2").collect()
hashes2[1].asDict()['hashes_2']


23/04/16 19:38:09 WARN ExtractPythonUDFFromJoinCondition: The join condition:(<lambda>(hashes_1#6774, hashes_2#6786)#6798 >= 0.0) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


DenseVector([299057.0, 985520.0, 4359465.0, 3554831.0, 40106801.0, 5005073.0, 3707795.0, 17026072.0, 4621514.0, 7956059.0, 130846.0, 11469437.0, 180588.0, 67344346.0, 3885125.0, 555686.0, 11468068.0, 2867121.0, 5580124.0, 14016668.0, 5585575.0, 3052659.0, 7009125.0, 11173503.0, 2830370.0, 4536514.0, 10265911.0, 24707324.0, 644522.0, 230594.0, 5161870.0, 11096079.0, 26826808.0, 32285854.0, 1964968.0, 880071.0, 35381907.0, 7729351.0, 23527797.0, 5912542.0, 25187536.0, 1151991.0, 25319454.0, 3144207.0, 45399513.0, 11463558.0, 20354358.0, 4216555.0, 17158323.0, 7279973.0, 438294.0, 1860153.0, 2206558.0, 300045.0, 6713918.0, 2398697.0, 5726783.0, 33267122.0, 267146.0, 2007772.0, 8817605.0, 3077162.0, 1793711.0, 4225102.0, 2817984.0, 1726167.0, 1216805.0, 12198856.0, 12100834.0, 18076318.0, 68436466.0, 9020725.0, 11240095.0, 16431510.0, 3747949.0, 8827840.0, 5707607.0, 22997035.0, 12579502.0, 21092182.0, 5582362.0, 15216095.0, 22498605.0, 1727060.0, 14005110.0, 1613063.0, 1442348.0, 11516432